<a href="https://colab.research.google.com/github/hatopopvr/MyBSTile_for_BeatLeader/blob/main/MyBSTile_for_BeatLeader_En.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MyBSTile_for_BeatLeader_En
<b>Data</b>
- Score Data form BeatLeader-Server v1 API - [doc](https://api.beatleader.xyz/swagger/index.html)  
- Cover Image from - https://eu.cdn.beatsaver.com/{hash}.jpg
- Player Image from - https://cdn.beatleader.xyz/assets/{player_id}.png

<b>Author</b>
- hatopop ([@hatopop_vr](https://twitter.com/hatopop_vr))

In [5]:
#@title <h1>Input & GetData from BeatLeader</h1> { display-mode: "form" }
#@markdown #### please input information
#@markdown ---
#@markdown <font size="2">`player_id` : PlayerID for BeatLeader(=ScoreSaberID). <b>Must</b> be changed.<br/></font>
player_id =  76561198412839195#@param {type:"number"}

#markdown ---
#markdown <h4>Setting</h4>
#markdown <font size="2">`ss_plus_is_enable`: AccRank区分にSS+を使用するか。☑で使用。  </font><br />
ss_plus_is_enable = True #param {type:"boolean"}
#markdown <font size="2">`ss_plus_val`:SS+の設定値です。必要に応じて変更ください。範囲は91-99。
ss_plus_val = 95 #param {type:"slider", min:91, max:99, step:1}
#@markdown <font size="2">`score_count`:The number of scores to retrieve from the BeatLeader-Server v1 API.
score_count = 50 #@param {type:"integer"}

#@markdown <font size="2">`sort_type`:Sort key before retrieving data</font>
#markdown <font size="2">`view_score_type`:The target pages to be retrieved by ScoreSaber API. `top`:`Top Scores`, `recent`:`Recent Scores`
sort_type = "pp" #@param ["date", "pp", "acc", "rank", "stars"]
#@markdown <font size="2">`order`: Descending(`desc`) or Ascending(`asc`) order
order = "desc" #@param ["desc", "asc"]
#markdown ---
#markdown <h4>Filters</h4>
stars_from = 0 #param {type:"slider", min:0, max:15, step:1}
stars_to = 15 #param {type:"slider", min:0, max:15, step:1}
song_keyword_filter = "" #param {type:"string"}

#@markdown ---
#@markdown <h4>Option</h4>

#@markdown <font size="2">`image_url_type`:URL redirected when the image is clicked.</font>
image_url_type = "leaderboard" #@param ["leaderboard", "replay"]
#@markdown <font size="2">`player_visible`:Whether to include player images in tiles.</font>
player_visible = False #@param {type:"boolean"}

# ----------------------------------------
# Import Library
# ----------------------------------------
#!pip install GitPython
import warnings
warnings.filterwarnings("ignore")
import os
import shutil
import pandas as pd
from pandas import json_normalize
import json
from datetime import datetime, timedelta
from dateutil import tz
import time
from tqdm import tqdm
import requests
import math
from IPython.display import HTML, Javascript,Image
import numpy as np
from PIL import Image
import copy


# ----------------------------------------
# Time Zone Set
# ----------------------------------------
# Timezone UTC->日本時間+9:00での実行日時の取得(tz_ja)
tz_ja = pd.Timestamp(datetime.now()).tz_localize('UTC').tz_convert('Asia/Tokyo')

# ----------------------------------------
# Parameter Setting
# ----------------------------------------
# SS_plus設定値
ss_plus = "SS+{}".format(ss_plus_val)
ss_plus_rate = "SS+{}-Rate".format(ss_plus_val)
# その他 colab表示など設定
pd.options.display.precision = 2
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# 出力最大高さ
def resize_colab_cell():
    display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'))

get_ipython().events.register('pre_run_cell', resize_colab_cell)

# ----------------------------------------
# Columns Setting
# ----------------------------------------

cover_image_size=70 #カバー,プロファイル画像のサイズ単位px
info_image_size=150 #Player画像のサイズ単位px

ss_plus = "SS+{}".format(ss_plus_val)
ss_plus_rate = "SS+{}-Rate".format(ss_plus_val)

# ----------------------------------------
# Function Color Style Setting
# ----------------------------------------
# Difficulty color
def color_difficulty(val):
    if val == "Easy":
        return 'color: #66BB6A; font-weight: bold;'
    elif val == "Normal":
        return 'color: #29B6F6; font-weight: bold;'
    elif val == "Hard":
        return 'color: #FB8C00; font-weight: bold;'
    elif val == "Expert":
        return 'color: #E53935; font-weight: bold;'
    elif val == "Expert+":
        return 'color: #8E24AA; font-weight: bold;'
    elif val == "ExpertPlus":
        return 'color: #8E24AA; font-weight: bold;'
    return 'color: #black'

# Acc Rank color
def color_acc_rank(val):
    if val == "SSS":
        return 'color: #00ffff; font-weight: bold;'
    elif ss_plus_is_enable and val == ss_plus:
        return 'color: #636EFA; font-weight: bold;'
    elif val == "SS":
        return 'color: #ff4500; font-weight: bold;'
    elif val == "S":
        return 'color: #ffaaff; font-weight: bold;'
    elif val == "A":
        return 'color: #ffcc66; font-weight: bold;'
    elif val == "B":
        return 'color: #cccc66; font-weight: bold;'
    else:
        return 'color: #666666; font-weight: bold;'

# FC color
def color_fc(val):
    if val == "FC":
        return 'color: #23D160; font-weight: bold;'
    else:
        return 'color: black'

# pp color
def color_pp(val, target_pp):
    if val >= target_pp:
        color = "red"
    else:
        color = "black"
    return 'color: %s' % color


# pp color
def color_pp(x):
    if  x >= 600: # SSS
        color = "#00ffff"
    elif x >= 500: # SS+
        color = "#636EFA"
    elif x >= 400: # SS
        color = "#ff4500"
    elif x >= 300: # S
        color = "#ffaaff"
    elif x >= 250: # A
        color = "#ffcc66"
    elif x >= 200: # B
        color = "#cccc66"
    else:
        color = "black"
    return 'color: %s' % color

def color_player_pp(x):
    if  x >= 16000:
        color = "#00ffff"
    elif x >= 14000:
        color = "#636EFA"
    elif x >= 12000:
        color = "#ff4500"
    elif x >= 10000:
        color = "#ffaaff"
    elif x >= 8000:
        color = "#ffcc66"
    elif x >= 6000:
        color = "#cccc66"
    elif x >= 4000:
        color = "#666666"
    # elif x >= 2000:
    #     color = "#994400"
    else:
        color = "black"
    return 'color: %s' % color

def color_player_rank(x):
    if  x <= 10: # Top10
        color = "#00ffff"
    elif x <= 50: # 1ページ目
        color = "#636EFA"
    elif x <= 100: # 2ページ目
        color = "#ff4500"
    elif x <= 500:
        color = "#ffaaff"
    elif x <= 1000:
        color = "#ffcc66"
    elif x <= 3000:
        color = "#cccc66"
    elif x <= 5000:
        color = "#cccc66"
    elif x <= 10000:
        color = "#666666"
    else:
        color = "black"

def color_rank(x):
    if  x <= 10: # Top10
        color = "#00ffff"
    elif x <= 50: # 1ページ目
        color = "#636EFA"
    elif x <= 100: # 2ページ目
        color = "#ff4500"
    elif x <= 500:
        color = "#ffaaff"
    elif x <= 1000:
        color = "#ffcc66"
    elif x <= 3000:
        color = "#cccc66"
    elif x <= 5000:
        color = "#cccc66"
    elif x <= 10000:
        color = "#666666"
    else:
        color = "black"
    return 'color: %s' % color

# ----------------------------------------
# Function Pandas Apply
# ----------------------------------------
def func_unrank(x):
    if x == 0:
        return None
    else:
        return x

def func_accrank_format(x):
    if  x > 100:
        return "-"
    elif  x == 100:
        return "SSS"
    elif x >= ss_plus_val and ss_plus_is_enable:
        return ss_plus
    elif x >= 90:
        return "SS"
    elif x >= 80:
        return "S"
    elif x >= 65:
        return "A"
    elif x >= 50:
        return "B"
    elif x >= 35:
        return "C"
    elif x >= 20:
        return "D"
    elif x >= 0:
        return "E"
    else:
        return "-"

def func_difficulty_format(val):
    if val == "Easy":
        return 'E'
    elif val == "Normal":
        return 'N'
    elif val == "Hard":
        return 'H'
    elif val == "Expert":
        return 'Ex'
    elif val == "Expert+":
        return 'Ex+'
    elif val == "ExpertPlus":
        return 'Ex+'
    return '-'

def func_fc_format(val):
    if val == "FC":
        return 'FC'
    else:
        return ''

# weight
def format_weight(val):
    if val <= target_weight:
        return "".format(val)
    else:
        return "({:.1f}pp)".format(val)

# star
def format_star(val):
    if val > 0:
        return "{:.2f}★".format(val)
    else:
        return ""
    return "{:.2f}★".format(val)

# acc
def format_acc(val):
    if val == 0:
        return ""
    elif val <= 100:
        return "{:.2f}%".format(val)
    else:
        return ""
    return "{:.2f}%".format(val)

# pp
def format_pp(val):
    if val > 0:
        return "{:.1f}pp".format(val)
    else:
        return ""
    return "{:.1f}pp".format(val)

def format_ranked(x):
    if x > 0:
        return True
    else:
        return False

# ----------------------------------------
# Player Image Css
# ----------------------------------------
def func_tile_css(_width=1000,
             _cover_image_size=200,
             _star_font_size=22,
             _diff_font_size=18,
             _acc_font_size=18,
             _fc_font_size=18,
             _pp_font_size=22,
             _rank_font_size=22,
             _player_font_size=22
             ):
    return """
        <style>
        .flex_box {
            width:""" + str(_width) + """px;
            display: flex; /* フレックスボックスにする */;
            flex-wrap: wrap;
            padding:0px;
            margin:0px;
        }
        .flex_item {
            display: inline-block;
            position: relative;
            padding:0px 5px 0px 0px;
            margin:0px;
        }
        .flex_str_top, .flex_str_bottom_left, .flex_str_bottom_right{
            position: absolute;
            font-weight: bold;
            white-space:nowrap;
            overflow:hidden;
            color: #000;
            width:""" + str(_cover_image_size-10) + """px;
            text-shadow:1px 1px 0 #FFF, -1px -1px 0 #FFF,
                    -1px 1px 0 #FFF, 1px -1px 0 #FFF,
                    0px 1px 0 #FFF,  0-1px 0 #FFF,
                    -1px 0 0 #FFF, 1px 0 0 #FFF;
        }

        .flex_str_top {
            left: 5px;
            top: 5px;
        }

        .flex_str_bottom_left {
            text-align:right;
            left: 5px;
            bottom: 5px;
        }

        .flex_str_bottom_right {
            left: 5px;
            bottom: 5px;
        }

        .flex_star, .flex_acc, .flex_accrank, .flex_fc ,.flex_pp, .flex_weight {
            padding:0px;
            margin:0px;
            text-align:right;
        }

        .flex_star{
            font-size: """ + str(_star_font_size) + """px;
        }

        .flex_diff{
            font-size: """ + str(_diff_font_size) + """px;
        }

        .flex_acc{
            font-size: """ + str(_acc_font_size) + """px;
        }

        .flex_fc {
            font-size: """ + str(_fc_font_size) + """px;
        }

        .flex_pp{
            font-size: """ + str(_pp_font_size) + """px;
        }

        .flex_rank {
            padding:0px;
            margin:0px;
            font-size: """ + str(_rank_font_size) + """px;
        }

        .flex_player{
            font-size: """ + str(_player_font_size) + """px;
        }
        </style>
        """
# ----------------------------------------
# Player html
# ----------------------------------------
def func_player(_cover_image_size=200):
    div_img = "<a href='https://www.beatleader.xyz/u/{}' target='_blank'><img src='{}' style='width:{}px;height:{}px'/></a>".format(player_id, df_info["avatar"][0],_cover_image_size, _cover_image_size)

    div_name = "<div class='flex_player'>{}</div>".format(df_info["name"][0])

    div_playcount = "<div class='flex_song'>TotalPlayCount:{:.0f}</div>".format(df_info["TotalPlayCount"][0])
    div_rankedplaycount = "<div class='flex_song'>RankedPlayCount:{:.0f}</div>".format(df_info["RankedPlayCount"][0])
    div_playcount = "<div class='flex_song'>RankedPlay:{:.0f}(Total:{:.0f})</div>".format(df_info["RankedPlayCount"][0],df_info["TotalPlayCount"][0])

    span_acc = "<span class='flex_acc' style='{}'>{}</span>".format(color_acc_rank(df_info['AverageRankedAccRank'][0]), format_acc(df_info["AverageRankedAcc"][0]*100))
    div_acc = "<div>{}</div>".format(span_acc)

    span_pp = "<span class='flex_pp' style='{}'>{}</span>".format(color_player_pp(df_info["pp"][0]),(format_pp(df_info["pp"][0])))
    div_pp = "<div>{}</div>".format(span_pp)
    div_country_rank = "<div class='flex_rank' style='{}'>#{:,.0f}[{}]</div>".format(color_player_rank(df_info["countryRank"][0]), df_info["countryRank"][0], df_info["country"][0])
    div_rank = "<div class='flex_rank' style='{}'>#{:,.0f}</div>".format(color_player_rank(df_info["rank"][0]), df_info["rank"][0])
    return """
            <div class='flex_item'>{}<br/>
                <div class='flex_str_top'>
                    {}
                    {}
                </div>
                <div class='flex_str_bottom_left'>
                    {}
                    {}
                </div>
                <div class='flex_str_bottom_right'>
                    {}
                    {}
                </div>
            </div>
            """.format(div_img, div_name, div_playcount, div_acc, div_pp, div_country_rank, div_rank)


# ----------------------------------------
# Get Score Data from BeatLeader
# ----------------------------------------
# Score情報の取得 ---------------------------------------
url = 'https://api.beatleader.xyz/player/{}/scores?sortBy={}&order={}&page=1&count={}&search={}&stars_from={}&stars_to={}'.format(player_id, sort_type, order, score_count, song_keyword_filter, stars_from, stars_to)

#stars_from
response = requests.get(url)
res_data = response.json()
df_scores = json_normalize(res_data['data'])

if len(df_scores) > 0:
    df_scores['Song'] = df_scores['leaderboard.song.name'] + " " + df_scores['leaderboard.song.subName'] + " / " + df_scores['leaderboard.song.author'] + " [" + df_scores['leaderboard.song.mapper'] + "]"
    df_scores['SongName'] = df_scores['leaderboard.song.name']
    df_scores['SongSub'] = df_scores['leaderboard.song.subName']
    df_scores['SongAuthor'] = df_scores['leaderboard.song.author']
    df_scores['LevelAuthor'] = df_scores['leaderboard.song.mapper']
    df_scores['Hash'] = df_scores['leaderboard.song.hash'].str.upper()
    df_scores['Acc'] = df_scores['accuracy'] * 100 # / df_scores['leaderboard.difficulty.maxScore'] * 100
    df_scores['MaxScore'] = df_scores['leaderboard.difficulty.maxScore']
    df_scores['Mode'] = df_scores['leaderboard.difficulty.modeName']#.apply(func_mode)
    df_scores['Difficulty'] = df_scores['leaderboard.difficulty.difficultyName']
    df_scores['Stars'] = df_scores['leaderboard.difficulty.stars']
    df_scores['Score'] = df_scores['modifiedScore']
    df_scores['Bad'] = df_scores['badCuts']
    df_scores['Miss'] = df_scores['missedNotes']
    df_scores['PP'] = df_scores['pp']
    df_scores['PPWeight'] = df_scores['bonusPp']# * df_scores['score.weight']
    df_scores['Rank'] = df_scores['rank']
    df_scores['Modifiers'] = df_scores['modifiers']
    df_scores['ReplayUrl'] = "https://replay.beatleader.xyz/?scoreId=" + df_scores['id'].astype('str')
    df_scores['LeaderboardUrl'] = "https://www.beatleader.xyz/leaderboard/global/" + df_scores['leaderboardId'].astype('str')
    df_scores['Ranked'] = df_scores['Stars'].apply(format_ranked)
    df_scores['AccRank'] = df_scores['Acc'].apply(func_accrank_format)
    df_scores['Cover'] = '<img src="'+df_scores['leaderboard.song.coverImage']+'" style="width:{}px;"/>'.format(cover_image_size)
    df_scores['CoverUrl'] = df_scores['leaderboard.song.coverImage']
    df_scores['FC'] = df_scores['fullCombo'].apply(func_fc_format)
    df_scores = df_scores[[x for x in df_scores.columns if not x.startswith("scoreImprovement.")]]
    df_scores = df_scores[[x for x in df_scores.columns if not x.startswith("leaderboard.")]]

    # 改行コード等の除去
    for col in df_scores.columns:
        try:
            if len(df_scores[df_scores[col].str.contains("\n")][[col]]) == 0:
                continue
            else:
                df_scores[col] = df_scores[col].str.replace("\n","")
        except:
            continue

    for col in df_scores.columns:
        try:
            if len(df_scores[df_scores[col].str.contains("\r")][[col]]) == 0:
                continue
            else:
                df_scores[col] = df_scores[col].str.replace("\r","")
        except:
            continue


# ----------------------------------------
# Get Player Data from BeatLeader
# ----------------------------------------
url = r"https://api.beatleader.xyz/player/{}?stats=true".format(player_id)
response = requests.get(url)
res_data = response.json()
df_info = json_normalize(res_data)

df_info["Pic"] = '<img src="'+df_info["avatar"]+'" style="width:{}px;"/>'.format(info_image_size)
df_info["TotalPlayCount"] = df_info["scoreStats.totalPlayCount"]
df_info["RankedPlayCount"] = df_info["scoreStats.rankedPlayCount"]
df_info["AverageRankedAcc"] = df_info["scoreStats.averageRankedAccuracy"]
df_info['AverageRankedAccRank'] = func_accrank_format(df_info["AverageRankedAcc"][0]*100)


str_player_info = """
{} results retrieved
""".format(len(df_scores))
print(str_player_info)

img_css_player = func_tile_css()

player_img = func_player() + img_css_player
HTML(player_img)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


50 results retrieved



# History Tile

In [6]:
#@title ## History Tile

#markdown ---
#markdown <h4>Enable</h4>

history_tile_is_enable = True #param {type:"boolean"}

#@markdown ---
#@markdown <h4>Setting</h4>

data_max_size = 500 #param {type:"slider", min:0, max:5000, step:1}

#@markdown <b>image setting</b>
#@markdown <font size=2> `width` : The width of the frame in px. Set to match (slightly larger than) the size of the image.
width = 1000 #@param {type:"slider", min:200, max:4000, step:100}
#@markdown <font size=2> `cover_image_size` : Width and height in px per image.
cover_image_size = 200 #@param {type:"slider", min:50, max:1000, step:50}

#@markdown <b>font size setting</b>
#@markdown <font size=2> : The setting of each font size.</font>
song_font_size = 22 #@param {type:"slider", min:3, max:50, step:1}
star_font_size = 18 #@param {type:"slider", min:3, max:50, step:1}
diff_font_size = 18 #@param {type:"slider", min:3, max:50, step:1}
acc_font_size = 18 #@param {type:"slider", min:3, max:50, step:1}
fc_font_size = 18 #@param {type:"slider", min:3, max:50, step:1}
pp_font_size = 22 #@param {type:"slider", min:3, max:50, step:1}
weight_font_size = 14 #@param {type:"slider", min:3, max:50, step:1}
rank_font_size = 22 #@param {type:"slider", min:3, max:50, step:1}
player_font_size = 22 #@param {type:"slider", min:3, max:50, step:1}

#@markdown <b>Visible Setting</b>
#@markdown <font size=2> : The setting of show/hide each item.</font>

song_visible = True #@param {type:"boolean"}
star_visible = True #@param {type:"boolean"}
diff_visible = True #@param {type:"boolean"}
acc_visible = True #@param {type:"boolean"}
fc_visible = True #@param {type:"boolean"}
pp_visible = True #@param {type:"boolean"}
weight_visible = True #@param {type:"boolean"}
rank_visible = True #@param {type:"boolean"}

#@markdown ---
#@markdown <h4>Filters</h4>
#@markdown <font size=2> Each filter function. Keyword filters do not support multiple keywords.</font>
filtered_pp_min = 0 #@param {type:"slider", min:0, max:1000, step:10}
filtered_pp_max = 1000 #@param {type:"slider", min:0, max:1000, step:10}
filtered_acc_min = 0 #@param {type:"slider", min:0, max:100, step:1}
filtered_acc_max = 100 #@param {type:"slider", min:0, max:100, step:1}
filtered_ranked_is_enable = False #@param {type:"boolean"}
filtered_fullcombo_is_enable = False #@param {type:"boolean"}

def func_flex(x):
    # div song
    div_song = "<div class='flex_song'>{}</div>".format(x[1]) if song_visible else ""

    # div star-diff
    span_star = "<span class='flex_star'>{}</span>".format(format_star(x[2])) if star_visible else ""
    span_diff = "<span= class='flex_diff'>{}</span>".format(func_difficulty_format(x[6])) if diff_visible else ""
    div_star_diff = "<div style='{}'>{} {}</div>".format(color_difficulty(x[6]), span_diff, span_star)

    # div acc-fc
    span_acc = "<span class='flex_acc' style='{}'>{}</span>".format(color_acc_rank(x[4]), format_acc(x[3])) if acc_visible else ""
    span_fc = "<span class='flex_fc' style='{}'>{}</span>".format(color_fc(x[5]),func_fc_format(x[5])) if fc_visible else ""
    div_acc_fc = "<div>{} {}</div>".format(span_fc, span_acc)

    # div pp
    span_pp = "<span class='flex_pp' style='{}'>{}</span>".format(color_pp(x[7]), format_pp(x[7])) if pp_visible else ""
    div_pp_weight = "<div>{}</div>".format(span_pp)#, span_weight)

    # div rank
    div_rank = "<div class='flex_rank' style='{}'>#{:,.0f}</div>".format(color_rank(x[9]), x[9]) if rank_visible else ""

    # div unrank
    span_diff_unrank = "<span= class='flex_star' style='{}'>{}</span>".format(color_difficulty(x[6]), func_difficulty_format(x[6])) if diff_visible else ""
    div_diff_fc = "<div style='{}'>{} {}</div>".format(color_difficulty(x[6]), span_diff, span_fc)
    return """
            <div class='flex_item'>{}<br/>
                <div class='flex_str_top'>
                    {}
                </div>
                <div class='flex_str_bottom_left'>
                    {}
                    {}
                    {}
                </div>
                <div class='flex_str_bottom_right'>
                    {}
                </div>
            </div>
            """.format(x[0], div_song, div_star_diff, div_acc_fc, div_pp_weight, div_rank)


# ----------------------------------------
# Create Tile
# ----------------------------------------

if len(df_scores) > 0:
    df_history = df_scores[(1==1)
        #& (df_scores["Modifiers"] != "NF")
        & (df_scores["PP"] >= filtered_pp_min)
        & (df_scores["PP"] <= filtered_pp_max)
        & (df_scores["Acc"] >= filtered_acc_min)
        & (df_scores["Acc"] <= filtered_acc_max)
        | (df_scores['Ranked']!=1)
        ]

    if filtered_ranked_is_enable:
        df_history = df_history[(df_history['Ranked']==1)]

    if filtered_fullcombo_is_enable:
        df_history = df_history[(df_history['FC']=="FC")]

    if len(song_keyword_filter) > 0:
        df_history = df_history[df_history["Song"].str.contains(song_keyword_filter, case=False)]

    history_rows_count = len(df_history)

    if history_tile_is_enable:
        print("{} results retrieved.\n".format(history_rows_count, data_max_size))
    else:
        print("有効フラグが無効です。")

    if history_rows_count > 0 and history_tile_is_enable:

        df_history_table = df_history.copy()

        df_history_table['Song'] = "<b>" + df_history_table["SongName"] + " " + df_history_table["SongSub"] + " </b><br /> " + df_history_table["SongAuthor"] + "<br />[" + df_history_table["LevelAuthor"] + "]"

        df_history_table = df_history_table

        df_history_table['Stars'] = df_history_table['Stars'].apply(func_unrank)

        df_flex_table = df_history_table.copy()

        if image_url_type == "leaderboard":
            df_flex_table["FlexCover"] = '<a href="'+ df_flex_table["LeaderboardUrl"] + '" target="_blank"><img src="' + df_flex_table["CoverUrl"] + '" style="width:{}px;height:{}px"/></a>'.format(cover_image_size,cover_image_size)
        elif image_url_type == "replay":
            df_flex_table["FlexCover"] = '<a href="'+ df_flex_table["ReplayUrl"] + '" target="_blank"><img src="' + df_flex_table["CoverUrl"] + '" style="width:{}px;height:{}px"/></a>'.format(cover_image_size,cover_image_size)
        else:
            df_flex_table["FlexCover"] = '<img src="' + df_flex_table["CoverUrl"] + '" style="width:{}px;height:{}px"/>'.format(cover_image_size,cover_image_size)

        df_flex_table["FlexItems"] =df_flex_table[['FlexCover','Song','Stars','Acc','AccRank','FC','Difficulty','PP', 'PPWeight','Rank','Ranked']].apply(func_flex, axis=1)

        # --------------------
        flex_head = """
                    <div id="imageDIV"
                        class='flex_box_wrapper'>
                        <div class='flex_box'>
                    """
        func_player = func_player() if player_visible else ""
        flex_content = " ".join(df_flex_table.reset_index()['FlexItems'].to_list())
        flex_end = "</div></div>"

        img_css = func_tile_css(width, cover_image_size, star_font_size,
                                diff_font_size, acc_font_size, fc_font_size,
                                pp_font_size, rank_font_size, player_font_size
                                )

        html_str = flex_head + func_player + flex_content + flex_end + img_css


    else:
        html_str = ""
else:
    html_str = ""

HTML(html_str)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

50 results retrieved.

